<a href="https://colab.research.google.com/github/theneuralcraftsman/disaster-tweet-predictor-nlp/blob/main/08_Introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence (seq2seq).

## Check for GPU

In [ ]:
!nvidia-smi

Mon Feb 19 16:42:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-02-19 16:42:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-02-19 16:42:59 (12.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

See the original source here: https://www.kaggle.com/competitions/nlp-getting-started/data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-02-19 16:43:03--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   837KB/s    in 0.7s    

2024-02-19 16:43:04 (837 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Pyhton: https://realpython.com/read-write-files-python/

But I prefer to get visual straight away.

So another way to do this is to use pandas....

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
from numpy.random import random_integers
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # Create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Jacksonville Fire &amp; Rescue Engine 58 - AWESOME TRUCK! - OFFICIAL JADE FL... https://t.co/ybdSlJw7c1 via @YouTube

---

Target: 0 (not real disaster)
Text:
how will I survive without dorrian

---

Target: 1 (real disaster)
Text:
Governor Allows Parole for School Bus Hijacker http://t.co/u4bdy1W7d4

---

Target: 1 (real disaster)
Text:
You can never escape me. Bullets don't harm me. Nothing harms me. But I know pain. I know pain. Sometimes I share it. With someone like you.

---

Target: 0 (not real disaster)
Text:
@news4buffalo yes a lot more harm then good if there are guardrails up now why cant we go 50. Their will be a big problem when school starts

---



### Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                                              train_df_shuffled["target"].to_numpy(),
                                                                                              test_size = 0.1, # use 10% of training data for validation split
                                                                                              random_state = 42)

In [ ]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to a number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned).

### Text vectorization (tokenization)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vacabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a tweet does a model see?)

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabullary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # Creates groups of n-words
                                    output_mode = "int", # How to map tokens into numbers,
                                    output_sequence_length=max_length, # how long do you want your sequences to be
                                    pad_to_max_tokens = True)

In [ ]:
# Splitting sentences in python
train_sentences[0].split(), len(train_sentences[0].split())

(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [ ]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentences = "There's a flood in my street!"
text_vectorizer([sample_sentences])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version: ")
text_vectorizer([random_sentence])

Original text:
 Police Monitoring 200 Jihadis in Poland http://t.co/1wCOfmLUb9 via @freedomoutpost      

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  77, 5098, 4184,    1,    4, 3529,    1,   49,    1,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()   # get all of the unique words in the text vectorizer
top_5_words = words_in_vocab[:5]  # get the most common words
bottom_5_words = words_in_vocab[-5:]   # get the least common words

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

To make our embedding, we're going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long.
* `input_length` = length of the sequences being paassed to the embedding layer.

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,  # set the input shape
                             output_dim = 128,  # choosing a value divisible by 8 increases the speed
                             embeddings_initializer='uniform',
                             input_length = max_length # How long is each input
                             )

embedding

In [ ]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version: ")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed


Original text:
 Rory McIlroy to Test Ankle Injury in Weekend Practice #chinadotcom #sports http://t.co/UDTGWfSc3P http://t.co/V5wSx0LQN2        

Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04460721, -0.00669348, -0.00946573, ...,  0.02999853,
          0.04870479, -0.01745725],
        [-0.03158925, -0.01460516,  0.02479465, ..., -0.02240368,
          0.04225147, -0.04702143],
        [ 0.03328006, -0.03468567,  0.00846536, ..., -0.03985833,
         -0.03225809, -0.01964383],
        ...,
        [-0.0040149 , -0.01126648,  0.02426123, ..., -0.0455138 ,
          0.01458423, -0.04266319],
        [ 0.0124218 ,  0.04315517,  0.01738926, ...,  0.00119916,
         -0.00333049, -0.01987101],
        [ 0.0124218 ,  0.04315517,  0.01738926, ...,  0.00119916,
         -0.00333049, -0.01987101]]], dtype=float32)>

In [ ]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04460721, -0.00669348, -0.00946573,  0.04003422, -0.02069548,
         0.03130317,  0.03117057, -0.01167352,  0.04585579,  0.03696373,
         0.03919882, -0.03828905,  0.03627614, -0.02644209, -0.0363924 ,
         0.00637366,  0.01004891,  0.01523776, -0.02978568, -0.00456791,
        -0.00741825, -0.03147378, -0.00544674,  0.01056863, -0.04231923,
         0.01242497, -0.00945222, -0.01863862, -0.03041452,  0.03592745,
         0.02641568,  0.02514629,  0.00987755, -0.01727744, -0.04347383,
         0.02538527, -0.03252144,  0.04759092,  0.03215465, -0.03386053,
        -0.02535863,  0.0398518 , -0.02221584,  0.02025688, -0.04683001,
        -0.0304335 ,  0.02260292,  0.02088196,  0.03066983,  0.00167668,
         0.02622775,  0.03999028, -0.00890125,  0.01289067, -0.03086917,
        -0.04876051,  0.04933972,  0.01410012, -0.04884342, -0.00328177,
        -0.02504175,  0.03530819,  0.00836942,  0.00299535, -0.02009823,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline).
* Model 1: Feed-forward neural network (dense model).
* Model 2: LSTM model (RNN).
* Model 3: GRU model (RNN).
* Model 4: Bidirectional-LSTM model (RNN).
* Model 5: 1D Convolutional Neural Network (CNN).
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data.


How are we going to approach all of these?

Use the standard steps in modelling with TensorFlow:

* Create a model
* Build a model
* Fit a model
* Evaluate a model


### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our texts to numbers.

>🔑**Note:** It's common practice to us non-DL algorithm as a baseline because of their speed and then later using DL to see if you can imporve upon them

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# Evaluation Metrics

* Accuracy - Default metrics for classification problems. Not the best for imbalanced classes.
* Precision - Higher precision leads to less false positives.
* Recall - Higher recall leads to less false negatives.
* F1-Score - Combination of precision and recall, usually a good overall metric for a clasification model.
* Confusion matrix - When comparing predictions to truth labels we see where model gets confused. Can be hard to use with large numbers of classes.

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and could easily be fixed with a function

Let's create one to compare our model's prediction with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-Score



In [ ]:
# Function to evaluate: accurate, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall, and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  # Calculate model precision, recall and f1-score using "weighted" average

  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}


  return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A simple dense model


In [ ]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [ ]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x)  # create an embedding of the numberized inputs
# Average the embeddings (flatten them and take the average)
x = layers.GlobalMaxPooling1D()(x) # Condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x)  # Create the output layer, want binary outputs so use sigmoid activation functions

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")


In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [ ]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
import numpy as np

In [ ]:
# # Assuming train_sentences and val_sentences are 1D arrays of strings
# train_sentences = np.array(train_sentences)
# val_sentences = np.array(val_sentences)

In [ ]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240219-164309
Epoch 1/5
215/215 [==============================] - 15s 59ms/step - loss: 0.6382 - accuracy: 0.6487 - val_loss: 0.5747 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.4676 - accuracy: 0.8288 - val_loss: 0.4713 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3377 - accuracy: 0.8758 - val_loss: 0.4508 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2549 - accuracy: 0.9085 - val_loss: 0.4525 - val_accuracy: 0.8005
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1949 - accuracy: 0.9349 - val_loss: 0.4654 - val_accuracy: 0.7966


In [ ]:
# Check the results
model_1.evaluate(val_sentences, val_labels)


24/24 [==============================] - 0s 8ms/step - loss: 0.4654 - accuracy: 0.7966


[0.4653955399990082, 0.7965879440307617]

In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [ ]:
# First 10 predictions
model_1_pred_probs[:10]

array([[0.4568451 ],
       [0.8743441 ],
       [0.9948996 ],
       [0.05636309],
       [0.26625955],
       [0.9774139 ],
       [0.9398418 ],
       [0.99248964],
       [0.941604  ],
       [0.07346085]], dtype=float32)

In [ ]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [ ]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.65879265091863,
 'precision': 0.7996948573021726,
 'recall': 0.7965879265091863,
 'f1': 0.7944470866850033}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([ True, False,  True,  True])

## Visualizing learned embeddings

In [ ]:
# Get the vocabulary from the text vactorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, Tensorflow has a handy tool called projector: https://projector.tensorflow.org/

Also Tensorflow themseleves have resources: https://www.tensorflow.org/text/guide/word_embeddings

In [ ]:
x# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights, embed_weights.shape # same size as vocab size and embedding_dim (output_dim of out embedding layer)

(array([[-0.08392613, -0.05079218, -0.02817629, ..., -0.09173187,
         -0.09468978, -0.03130464],
        [-0.08934513, -0.06108735, -0.02936928, ..., -0.093233  ,
         -0.09936564, -0.04266319],
        [-0.0288605 , -0.04727589, -0.02572581, ..., -0.04912325,
         -0.07002445, -0.02950043],
        ...,
        [ 0.0126985 , -0.02367579,  0.02492407, ...,  0.00047678,
         -0.02260824, -0.02796462],
        [-0.00798862, -0.02445419, -0.04043609, ..., -0.04787744,
          0.00949106, -0.0226931 ],
        [-0.0328893 ,  0.00175909, -0.04666392, ..., -0.04789338,
         -0.04282232,  0.09316183]], dtype=float32),
 (10000, 128))

In [ ]:
import io

In [ ]:
# Create embedding files (we got this from TensorFlow's word embeddings documentation)
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# Download files from Colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNNs)

RNNs are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

>📖 **Resources**:
- Overview of internals of RNN:
https://www.youtube.com/watch?v=ySEx_Bqxvvo
- Chris olah's intro to LSTM:
https://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Andrej Karpathy's the unreasonable Effectiveness of Recurrent Neural Networks:
https://karpathy.github.io/2015/05/21/rnn-effectiveness/

### Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (Label probability)


In [ ]:
# Create an LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # When ur stacking RNN cells together you need to set return sequences = True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)


In [ ]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Compile
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240219-164341
Epoch 1/5
215/215 [==============================] - 26s 97ms/step - loss: 0.3853 - accuracy: 0.8302 - val_loss: 0.4759 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.2440 - accuracy: 0.9066 - val_loss: 0.5310 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1763 - accuracy: 0.9362 - val_loss: 0.6198 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1336 - accuracy: 0.9542 - val_loss: 0.6452 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1020 - accuracy: 0.9628 - val_loss: 0.8178 - val_accuracy: 0.7822


In [ ]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 6ms/step


array([[0.01875581],
       [0.9098147 ],
       [0.9988285 ],
       [0.01046469],
       [0.00405624],
       [0.9994466 ],
       [0.9509304 ],
       [0.9996958 ],
       [0.9993917 ],
       [0.29822612]], dtype=float32)

In [ ]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 78.21522309711287,
 'precision': 0.7843528649827862,
 'recall': 0.7821522309711286,
 'f1': 0.7800522271093588}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [ ]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x) # If you want to stack recurrent layers on top of each other you want the return sequences to be true
# x = layers.LSTM(6, return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name = "model_3_GRU")

In [ ]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_8 (GRU)                 (None, 64)                37248     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [ ]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240219-172413
Epoch 1/5
215/215 [==============================] - 13s 47ms/step - loss: 0.2102 - accuracy: 0.9091 - val_loss: 0.6463 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1046 - accuracy: 0.9619 - val_loss: 0.7736 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0786 - accuracy: 0.9712 - val_loss: 0.9563 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0589 - accuracy: 0.9756 - val_loss: 1.1120 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0510 - accuracy: 0.9764 - val_loss: 1.2728 - val_accuracy: 0.7651


In [ ]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[2.7499463e-02],
       [7.3475820e-01],
       [9.9990106e-01],
       [1.4373104e-01],
       [2.4179902e-04],
       [9.9989450e-01],
       [6.3066655e-01],
       [9.9994946e-01],
       [9.9993134e-01],
       [9.7471243e-01]], dtype=float32)

In [ ]:
# Convert model_3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)

model_3_results, np.array(list(model_3_results.values())) > np.array(list(baseline_results.values()))

({'accuracy': 76.50918635170603,
  'precision': 0.7648935744284318,
  'recall': 0.7650918635170604,
  'f1': 0.7641581623335479},
 array([False, False, False, False]))

### Model_4 : Bidirectional-LSTM (RNN)

Normal RNN's go from left to right (just like you'd read an English sentences) however, a bidirectional RNN goes from right to left as well as left to right.

In [ ]:
# Build a bidirectional RNN in TensorFLow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)

x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [ ]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [ ]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_4.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences, val_labels),
            callbacks = [create_tensorboard_callback(SAVE_DIR, "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240219-181743
Epoch 1/5
215/215 [==============================] - 17s 58ms/step - loss: 0.1080 - accuracy: 0.9644 - val_loss: 1.0560 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0420 - accuracy: 0.9791 - val_loss: 1.2355 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0357 - accuracy: 0.9829 - val_loss: 1.2406 - val_accuracy: 0.7585
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0367 - accuracy: 0.9820 - val_loss: 1.4713 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0335 - accuracy: 0.9831 - val_loss: 1.3609 - val_accuracy: 0.7572


In [ ]:
# Make some predictions with our Bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[5.0085369e-02],
       [8.1559151e-01],
       [9.9998081e-01],
       [2.4750559e-01],
       [3.2420317e-05],
       [9.9990010e-01],
       [9.8857212e-01],
       [9.9999619e-01],
       [9.9999070e-01],
       [5.7588879e-02]], dtype=float32)

In [ ]:
# Convert model_4 pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)

model_4_results, np.array(list(model_4_results.values())) > np.array(list(baseline_results.values()))

({'accuracy': 76.50918635170603,
  'precision': 0.7648935744284318,
  'recall': 0.7650918635170604,
  'f1': 0.7641581623335479},
 array([False, False, False, False]))

## Convolution Neural Networks for Text (and other types of sequences)

We've used CNNs for images but images are typically 2D (height x width)... however, our text data is 1D.

Previously we've Conv2D for our image data but now we're going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (Class probabilities)
```

### Model 5: Conv 1D (1D used for text)

In [ ]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding